In [1]:
import numpy as np
import pandas as pd
from math import isnan
from sklearn import preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
import json
def importFile(file_path):

    #TODO : ability to set other parameters when loading data through constructor
    
    if file_path.endswith('.csv') or file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            first_line = f.readline()
        if '\t' in first_line:
            return pd.read_csv(file_path, sep='\t')
        elif ';' in first_line:
            return pd.read_csv(file_path, sep=',')
        elif ',' in first_line:
            return pd.read_csv(file_path, sep=',')
        else:
            return pd.read_fwf(file_path)

    elif file_path.endswith('.json'):
        with open(file_path) as json_file:
            data = json.load(json_file)
        return pd.DataFrame(data)

    elif file_path.endswith('.xlsx'):
        excel_file = pd.ExcelFile(file_path)
        sheets = excel_file.sheet_names
        number_of_sheets = len(sheets)
        print(f'found {number_of_sheets} sheets ')
        if number_of_sheets == 1:
            return pd.read_excel(file_path, sheet_name=sheets[0])
        else:
            df = []
            for i in range(number_of_sheets):
                df.append(pd.read_excel(file_path, sheet_name=sheets[i]))
            return df 

    # TODO connecting to database and load sql tables in dataframe
    elif file_path.endswith('.sql'):
        return pd.read_sql(file_path)

    else:
        # TODO return a message such that it doesn't abruptly stop the program (error handling)
        return None


In [7]:
df = importFile('./datasets/MarketSegmentation_Train.csv')
df.head(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [8]:
df.head(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [12]:
# Handling missing values

df.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [15]:
# drop row with null values
df = df.dropna(how='all') 
# reset the index of a pandas dataframe to default sequential values starting from 0.
# drop = True means that the old index is dropped and not included as a new column in the dataframe.
df = df.reset_index(drop=True)

In [16]:
df.select_dtypes(include=np.number).columns

Index(['ID', 'Age', 'Work_Experience', 'Family_Size'], dtype='object')

In [60]:
import copy
df1 = copy.deepcopy(df)

In [61]:
cols_num = df.select_dtypes(include=np.number).columns
mapping = dict()
for feature in df.columns:
    if feature not in cols_num:
        # create label mapping for categorical feature values
        mappings = {k: i for i, k in enumerate(df1[feature])}
        mapping[feature] = mappings
        df1[feature] = df1[feature].map(mapping[feature])

In [62]:
mapping

{'Gender': {'Male': 8067, 'Female': 8066},
 'Ever_Married': {'No': 8066, 'Yes': 8067, nan: 8044},
 'Graduated': {'No': 8064, 'Yes': 8067, nan: 7995},
 'Profession': {'Healthcare': 8066,
  'Engineer': 8048,
  'Lawyer': 8057,
  'Entertainment': 8058,
  'Artist': 8062,
  'Executive': 8067,
  'Doctor': 8047,
  'Homemaker': 8056,
  'Marketing': 8026,
  nan: 8063},
 'Spending_Score': {'Low': 8066, 'Average': 8067, 'High': 8062},
 'Var_1': {'Cat_4': 8067,
  'Cat_6': 8066,
  'Cat_7': 8020,
  'Cat_3': 8058,
  'Cat_1': 8063,
  'Cat_2': 8053,
  nan: 7991,
  'Cat_5': 7967},
 'Segmentation': {'D': 8065, 'A': 8060, 'B': 8067, 'C': 8061}}

In [53]:
mapping = {}
for i, k in enumerate(df1['Gender']):
    mapping[k] = i   #mappings['Male'] = 1....

{'Male': 8067, 'Female': 8066}


In [54]:
print({ k : i for i, k in enumerate(df['Gender']) })

{'Male': 8067, 'Female': 8066}


In [43]:
for feature in cols_num:    

    # dataframe of rows that have na values for that feature. All the rows containig na values for other features are dropped.
    test_df = df1[df1[feature].isnull()==True].dropna(subset=[x for x in df1.columns if x != feature])

    # dataframe of rows that do not have na values for that feature. All the rows containig na values for other features are dropped.
    train_df = df1[df1[feature].isnull()==False].dropna(subset=[x for x in df1.columns if x != feature])

In [44]:
len(test_df.index)

270

## encoding

In [29]:
#TODO ask if want to convert categorical to numerical
# categorical to numerical
df["Gender"] = df["Gender"].astype("category")
df["Gender_encoded"] = df["Gender"].cat.codes

## handling missing values

In [19]:
# mean imputation
df.mean()

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_16060\3977765826.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


ID                 463479.214551
Age                    43.466906
Work_Experience         2.641663
Family_Size             2.850123
dtype: float64

In [20]:
df.median()

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_16060\530051474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.median()


ID                 463472.5
Age                    40.0
Work_Experience         1.0
Family_Size             3.0
dtype: float64

In [22]:
df.mode().iloc[0]

ID                 458982
Gender               Male
Ever_Married          Yes
Age                  35.0
Graduated             Yes
Profession         Artist
Work_Experience       1.0
Spending_Score        Low
Family_Size           2.0
Var_1               Cat_6
Segmentation            D
Name: 0, dtype: object

In [ ]:
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer to the data and transform it
df_mean_imputed = pd.DataFrame(mean_imputer.fit_transform(df), columns=df.columns)
df_median_imputed = pd.DataFrame(median_imputer.fit_transform(df), columns=df.columns)
df_most_frequent_imputed = pd.DataFrame(most_frequent_imputer.fit_transform(df), columns=df.columns)

In [ ]:
knnimputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(knnimputer.fit_transform(df), columns=df.columns)

## Scaling

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler

# # Load the data set
# df = pd.read_csv("data.csv")

# # Create the rescaler object
# scaler = MinMaxScaler()

# # Fit the rescaler to the data and transform it
# df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
